This notebook creates the aggregated_data csv used in the Strong EHM test

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from google.colab import drive 
import os
import itertools
from tqdm import tqdm
drive.mount('/content/drive')
os.chdir('drive/MyDrive/SummerDO')

In [ ]:
accepted = pd.read_csv("Final Code Repo/Data/accepted_2007_to_2018Q4.csv").sample(frac = 1, axis = 0, random_state = 0)

In [ ]:
accepted = accepted[accepted['grade'].isna() == False]

In [ ]:
grades = sorted(accepted['grade'].unique())
accepted['term_y'] = [int(str(value)[:3])/12 for value in accepted['term']]

In [ ]:
grades = grades[:5]

In [ ]:
accepted['month'] = pd.PeriodIndex(accepted['issue_d'], freq='M')
accepted.set_index("month", inplace = True)
accepted = accepted.sort_values(by = 'month')

In [ ]:
accepted['int_rate']/= 100

In [ ]:
accepted['loan_status_ind'] = accepted["loan_status"].map({"Current" : -1, "Fully Paid" : 1, 
                                        "In Grace Period" : -1, "Charged Off" : 0, 
                                        "Default" : 0, "Late (16-30 days)" : 0, "Late (31-120 days)" : 0})

accepted = accepted[accepted['loan_status_ind'] != -1]


Actual_payoff = accepted['loan_amnt']*(1+accepted['int_rate']*accepted['term_y'])*(accepted['loan_status_ind'])
Actual_return = (100*(Actual_payoff - accepted['loan_amnt'])/accepted['loan_amnt'])/accepted['term_y']
accepted['ann_return'] = Actual_return

In [ ]:
accepted['real_payoff'] = accepted['loan_amnt']*(1+accepted['int_rate']*accepted['term_y'])*(accepted['loan_status_ind'])
accepted['real_return'] = (100*(accepted['real_payoff'] - accepted['loan_amnt'])/accepted['loan_amnt'])/accepted['term_y']

In [ ]:
accepted['loan_status_ind_neg'] = accepted['loan_status_ind'].replace({0:1, 1:0})

accepted['real_payoff_with_recoup'] = accepted['loan_amnt']*(1+accepted['int_rate']*accepted['term_y'])*(accepted['loan_status_ind']) + accepted['loan_status_ind_neg']*(accepted['total_rec_prncp'] + accepted['total_rec_int'])

accepted['real_return_with_recoup'] = (100*(accepted['real_payoff_with_recoup'] - accepted['loan_amnt'])/accepted['loan_amnt'])/accepted['term_y']


In [ ]:
accepted = accepted[(accepted.grade != 'F')&(accepted.grade != 'G')]

In [ ]:
accepted = accepted[~accepted['real_return_with_recoup'].isna()]

In [ ]:
accepted['month'] = pd.PeriodIndex(accepted['issue_d'], freq='M')
accepted.set_index("month", inplace = True)
accepted = accepted.sort_values(by = 'month')

In [ ]:
accepted_agg = (
    accepted
    .copy()
    .filter(['grade', 'issue_d', 'real_return_with_recoup', 'term_y'])
    .groupby(['grade', 'month']).mean()
    .reset_index()
)

accepted_agg.set_index("month", inplace = True)
accepted_agg = accepted_agg.sort_values(by = 'month')

In [ ]:
accepted_agg.to_csv("Aggregated_data.csv")